In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import tqdm
from sklearn.preprocessing import LabelEncoder
import tushare as ts
import warnings
pro = ts.pro_api('700c1d6015ad030ff20bf310c088243da030e6b79a2a1098d58d2614')

In [2]:
pro.fut_daily(ts_code = 'IC.CFX')

,ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg
0,IC.CFX,20250926,7166.6,7185.4,7147.0,7196.6,7080.0,7080.0,7097.6,-105.4,-87.8,82089.0,1.171498e+07,135754.0,None
1,IC.CFX,20250925,7157.0,7135.8,7139.0,7212.8,7132.4,7166.6,7185.4,30.8,49.6,76629.0,1.099618e+07,132726.0,None
2,IC.CFX,20250924,6940.6,6888.2,6910.0,7162.6,6906.0,7157.0,7135.8,268.8,247.6,103608.0,1.461418e+07,136396.0,None
3,IC.CFX,20250923,7013.2,6995.0,7033.0,7038.0,6789.0,6940.6,6888.2,-54.4,-106.8,107027.0,1.473969e+07,138800.0,None
4,IC.CFX,20250922,6984.0,6999.0,7007.0,7019.8,6925.2,7013.2,6995.0,14.2,-4.0,67975.0,9.486293e+06,128272.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,IC.CFX,20170714,6108.2,6105.8,6101.4,6125.0,6065.4,6093.4,6091.2,-12.4,-14.6,10337.0,1.260332e+06,17244.0,None
1996,IC.CFX,20170713,6085.4,6076.0,6100.0,6152.6,6080.4,6108.2,6105.8,32.2,29.8,13115.0,1.603498e+06,18118.0,None
1997,IC.CFX,20170712,6108.4,6117.4,6117.4,6123.8,6013.8,6085.4,6076.0,-32.0,-41.4,14774.0,1.792313e+06,19690.0,None
1998,IC.CFX,20170711,6165.0,6176.6,6173.0,6173.0,6092.2,6108.4,6117.4,-68.2,-59.2,11885.0,1.459100e+06,19294.0,None


In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
split = '20220101'
pred_len = 5

In [ ]:
from scipy.stats import norm
def process(data, pred_len):
    data['log_open'] = np.log(data['open'] / data['pre_close']) * 100 
    data['log_high'] = np.log(data['high'] / data['pre_close']) * 100 
    data['log_low'] = np.log(data['low'] / data['pre_close']) * 100 
    data['log_close'] = np.log(data['close'] / data['pre_close']) * 100
    data['log_amount'] = np.log(data['amount'] / data['amount'].shift(1)) * 10

    data['label_return'] = data['log_close'].rolling(window = pred_len).sum().shift(-pred_len)
    data['label_std'] = data['amount'].rolling(window = pred_len).mean().shift(-pred_len)/data['amount'].mean() * data['label_return'].std()

    threshold = (abs(data['label_return'].quantile(0.75)) + abs(data['label_return'].quantile(0.25)))/2
    print(threshold)

    def down_probability(row):
        return norm.cdf(-threshold, loc = row['label_return'], scale=row['label_std'])

    def middle_probability(row):
        return norm.cdf(threshold, loc = row['label_return'], scale=row['label_std']) - norm.cdf(-threshold, loc = row['label_return'], scale=row['label_std'])

    def up_probability(row):
        return 1 - norm.cdf(threshold, loc = row['label_return'], scale=row['label_std'])
    
    data['down_prob'] = data.apply(down_probability, axis = 1)
    data['middle_prob'] = data.apply(middle_probability, axis = 1)
    data['up_prob'] = data.apply(up_probability, axis = 1)
    data.dropna(inplace=True)
    
for index in ['000016.SH', '000300.SH', '000905.SH']:
    data = pro.index_daily(ts_code = index)
    process(data, pred_len = pred_len)
    train_data = data[data['trade_date']<split].copy()
    test_data = data[data['trade_date']>split].copy()

    train_feature = train_data[['log_open','log_high','log_low','log_close','log_amount']]
    train_label = train_data[['label_return','down_prob','middle_prob','up_prob']]

    test_feature = test_data[['log_open','log_high','log_low','log_close','log_amount']]
    test_label = test_data[['label_return','down_prob','middle_prob','up_prob']]


In [9]:
future_list = pro.fut_basic(fut_type='1',exchange = 'CFFEX', list_date = '20000101')
future_list

,ts_code,symbol,exchange,name,fut_code,multiplier,trade_unit,per_unit,quote_unit,quote_unit_desc,d_mode_desc,list_date,delist_date,d_month,last_ddate
0,T1906.CFX,T1906,CFFEX,T1906,T,10000.0,张,1.0,人民币元,0.005人民币元,实物交割,20180917,20190614,201906,20190619
1,TF1906.CFX,TF1906,CFFEX,TF1906,TF,10000.0,张,1.0,人民币元,0.005人民币元,实物交割,20180917,20190614,201906,20190619
2,IF1507.CFX,IF1507,CFFEX,IF1507,IF,300.0,张,1.0,指数点,0.2指数点,现金交割,20150518,20150717,201507,20150717
3,T1703.CFX,T1703,CFFEX,T1703,T,10000.0,张,1.0,人民币元,0.005人民币元,实物交割,20160614,20170310,201703,20170315
4,IM2505.CFX,IM2505,CFFEX,IM2505,IM,200.0,张,1.0,指数点,0.2指数点,现金交割,20250324,20250516,202505,20250516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,IC1601.CFX,IC1601,CFFEX,IC1601,IC,200.0,张,1.0,指数点,0.2指数点,现金交割,20151123,20160115,201601,20160115
624,IC2203.CFX,IC2203,CFFEX,IC2203,IC,200.0,张,1.0,指数点,0.2指数点,现金交割,20210719,20220318,202203,20220318
625,IF2203.CFX,IF2203,CFFEX,IF2203,IF,300.0,张,1.0,指数点,0.2指数点,现金交割,20210719,20220318,202203,20220318
626,IH2203.CFX,IH2203,CFFEX,IH2203,IH,300.0,张,1.0,指数点,0.2指数点,现金交割,20210719,20220318,202203,20220318


In [10]:
future_list = pro.fut_basic(fut_type='2')
future_list

,ts_code,symbol,exchange,name,fut_code,multiplier,trade_unit,per_unit,quote_unit,quote_unit_desc,d_mode_desc,list_date,delist_date,d_month,last_ddate
0,CFE_LL161MSF.WI,CFE_LL161MSF,None,上证50当月合成期货,HO.CFE,None,None,None,None,None,None,None,None,None,None
1,CFE_LL161QSF.WI,CFE_LL161QSF,None,上证50当季合成期货,HO.CFE,None,None,None,None,None,None,None,None,None,None
2,CFE_LL162MSF.WI,CFE_LL162MSF,None,上证50下月合成期货,HO.CFE,None,None,None,None,None,None,None,None,None,None
3,CFE_LL162QSF.WI,CFE_LL162QSF,None,上证50下季合成期货,HO.CFE,None,None,None,None,None,None,None,None,None,None
4,CFE_LL163MSF.WI,CFE_LL163MSF,None,上证50下下月合成期货,HO.CFE,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,IML3.CFX,IML3,CFFEX,中证1000期货下季连续,IM,None,None,None,None,None,None,None,None,None,None
207,TLL.CFX,TLL,CFFEX,30年期国债期货当季连续,TL,None,None,None,None,None,None,None,None,None,None
208,TLL1.CFX,TLL1,CFFEX,30年期国债期货下季连续,TL,None,None,None,None,None,None,None,None,None,None
209,TLL2.CFX,TLL2,CFFEX,30年期国债期货隔季连续,TL,None,None,None,None,None,None,None,None,None,None


In [11]:
pro.fut_daily(ts_code = 'IHL.CFX')

,ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg
0,IHL.CFX,20250925,2941.8,2939.2,2941.0,2965.0,2941.0,2953.8,2958.00,14.6,18.80,15211.0,1347711.984,20072.0,None
1,IHL.CFX,20250924,2923.2,2911.8,2916.8,2946.8,2915.0,2941.8,2939.20,30.0,27.40,18893.0,1662164.328,21098.0,None
2,IHL.CFX,20250923,2922.6,2917.0,2917.0,2937.0,2890.0,2923.2,2911.80,6.2,-5.20,21921.0,1913710.302,24243.0,None
3,IHL.CFX,20250922,2918.4,2917.4,2914.2,2930.0,2902.4,2922.6,2917.00,5.2,-0.40,15232.0,1332050.322,23325.0,None
4,IHL.CFX,20250919,2910.8,2907.8,2911.6,2929.8,2907.0,2918.4,2918.05,10.6,10.25,13660.0,1194631.044,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,IHL.CFX,20170713,2554.4,2552.8,2565.6,2598.4,2558.2,2597.6,2590.20,44.8,37.40,10111.0,782618.436,16115.0,None
1996,IHL.CFX,20170712,2567.0,2574.2,2566.6,2594.4,2544.2,2554.4,2552.80,-19.8,-21.40,12079.0,932068.104,16514.0,None
1997,IHL.CFX,20170711,2536.2,2536.4,2537.0,2586.6,2533.0,2567.0,2574.20,30.6,37.80,9687.0,744104.160,17359.0,None
1998,IHL.CFX,20170710,2523.4,2524.8,2518.2,2542.8,2516.2,2536.2,2536.40,11.4,11.60,7488.0,569168.802,16592.0,None


In [12]:
def update_data_tosql():
    
    '''
    交易所代码 CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心 GFEX-广州期货交易所
    '''

    future_list = pro.fut_basic(fut_type='1', list_date = '20000101')
    future_list = future_list.set_index('ts_code')


    for index in tqdm.tqdm(future_list.index):
        # print(index)

        while True:
            try:
                time.sleep(0.2)
                new_data = pro.fut_daily(ts_code = index)
                new_data = new_data.sort_values(by='trade_date')

                new_data.rename(columns = {
                    'pre_close' : 'price_close_lag1',
                    'pre_settle' : 'price_settle_lag1',
                    'open' : 'price_open',
                    'close' : 'price_close',
                    'high' : 'price_high',
                    'low' : 'price_low',
                    'settle' : 'price_settle',
                    'change1' : 'chg_close_close_lag1',
                    'change2' : 'chg_settle_close_lag1',
                    'oi_chg' : 'chg_oi_oi_lag1',
                }, inplace = True)

                for index in ['fut_code', 'list_date', 'delist_date', 'exchange']:
                    new_data[f'{index}'] = new_data['ts_code'].map(future_list[f'{index}'])

                # 相对价格

                for index in ['open', 'high', 'low', 'close', 'settle']:

                    new_data[f'chg_{index}_close_lag1'] = new_data[f'price_{index}'] - new_data['price_close_lag1']
                    new_data[f'pct_chg_{index}_close_lag1'] = new_data[f'chg_{index}_close_lag1'] / new_data['price_close_lag1']

                for index in ['high', 'low', 'close', 'settle']:
                    new_data[f'chg_{index}_open'] = new_data[f'price_{index}'] - new_data['price_open']
                    new_data[f'pct_chg_{index}_open'] = new_data[f'chg_{index}_open'] / new_data['price_open']



                # 历史数据信息
                for i in [5,10,20]:
                    new_data[f'chg_close_close_lag{i}'] = new_data['price_close'] - new_data['price_close'].shift(i)
                    new_data[f'pct_chg_close_close_lag{i}'] = new_data[f'chg_close_close_lag{i}'] / new_data['price_close'].shift(i)

                # 成交量、头寸
                new_data['pct_vol_overall'] = (new_data['vol'] - new_data['vol'].mean())
                new_data['pct_oi_overall'] = (new_data['oi'] - new_data['oi'].mean())


                # 转换日期格式
                new_data['trade_date'] = pd.to_datetime(new_data['trade_date'], format='%Y%m%d')
                new_data['list_date'] = pd.to_datetime(new_data['list_date'], format='%Y%m%d')
                new_data['delist_date'] = pd.to_datetime(new_data['delist_date'], format='%Y%m%d')

                # 计算相对日期、上市天数、剩余天数
                new_data['onlist_day'] = (new_data['trade_date'] - new_data['list_date']).dt.days
                new_data['maturity'] = (new_data['delist_date'] - new_data['trade_date']).dt.days
                new_data['date_count'] = (new_data['trade_date'] - datetime.datetime(2000,1,1,0,0,0)).dt.days

                new_data.to_sql(
                    name = 'panel_price',
                    con = engine,
                    if_exists = 'append',
                    index = False,
                    method = 'multi',
                )
                
                break

            except:
                pass
    
    

In [13]:
def update_data():
    
    '''
    交易所代码 CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心 GFEX-广州期货交易所
    '''

    data = pd.DataFrame(columns = ['ts_code', 'trade_date', 'pre_close', 'pre_settle', 'open', 'high', 'low', 'close', 'settle', 'change1', 'change2', 'vol', 'amount', 'oi', 'oi_chg'])

    future_list = pro.fut_basic(fut_type='1', list_date = '20150101')
    future_list = future_list.set_index('ts_code')


    for index in tqdm.tqdm(future_list.index):
        # print(index)

        while True:
            try:
                time.sleep(0.2)
                new_data = pro.fut_daily(ts_code = index)
                new_data = new_data.sort_values(by='trade_date')


                for index in ['fut_code', 'list_date', 'delist_date', 'exchange']:
                    new_data[f'{index}'] = new_data['ts_code'].map(future_list[f'{index}'])

                # 相对价格

                for index in ['open', 'high', 'low', 'close', 'settle']:
                    new_data[f'ch_{index}'] = (new_data[f'{index}'] - new_data['pre_close'])
                    new_data[f'pct_ch_{index}'] = new_data[f'ch_{index}'] / new_data['pre_close'] * 100
                    # 归一化

                    new_data[f'scaled_{index}'] = new_data[f'{index}'] / 76
                    new_data[f'scaled_ch_{index}'] = new_data[f'ch_{index}'] / new_data[f'ch_{index}'].std()

                # 历史数据信息
                for i in [5,10,20]:
                    new_data[f'pct_chg_day{i}'] = (new_data['pre_close'] - new_data['pre_close'].shift(i)) / new_data['pre_close'].shift(i) / i * 100
                # 成交量、头寸
                new_data['ch_vol'] = (new_data['vol'] - new_data['vol'].mean()) / 20000
                new_data['ch_oi'] = (new_data['oi'] - new_data['oi'].mean()) / 34000


                # 转换日期格式
                new_data['trade_date'] = pd.to_datetime(new_data['trade_date'], format='%Y%m%d')
                new_data['list_date'] = pd.to_datetime(new_data['list_date'], format='%Y%m%d')
                new_data['delist_date'] = pd.to_datetime(new_data['delist_date'], format='%Y%m%d')

                # 计算相对日期、上市天数、剩余天数
                new_data['onlist_day'] = (new_data['trade_date'] - new_data['list_date']).dt.days
                new_data['maturity'] = (new_data['delist_date'] - new_data['trade_date']).dt.days

                new_data['scaled_onlist_day'] = new_data['onlist_day'] / 100
                new_data['scaled_maturity'] = new_data['maturity'] / 100

                new_data['date_count'] = (new_data['trade_date'] - datetime.datetime(2015,1,1,0,0,0)).dt.days

                

                data = pd.concat([data, new_data])
                break

            except:
                pass
    
    data.sort_values(by = 'date_count', inplace = True)
    data.dropna(inplace=True)

    label_encoder = LabelEncoder()
    data['fut_code_num'] = label_encoder.fit_transform(data['fut_code'])

    

    data.to_csv('data/futures_data.csv', index = False)
    

In [14]:
data_set = pd.read_csv('data/futures_data.csv')
data_set

FileNotFoundError: [Errno 2] No such file or directory: 'data/futures_data.csv'